# Facade Tiling

__Project Concept__

The case study is a façade envelope for an office building in Av. República, Lisbon, Portugal. The façade design is inspired by triangular tiling techniques. Tiling refers to any partition of the plane into smaller regions, named tiles, that fit together without gaps or overlaps. This technique has found several applications in architecture, namely, to explore both the tectonic and visual expressiveness of tiling to generate building skins with multiple geometric patterns and performance behaviors (e.g., shading, ventilation, lighting proprieties, among others). In this notebook, we explore the geometric development of an algorithmic triangular tiling façade: we use tiling to create a visually interesting/dynamic geometric pattern, as well as to shade the interior spaces of the building from the intense natural daylight typical of southern European countries.

__Notebook Structure__

The notebook is organized according to the development stages of the project, including the set up of the site, the exploration of the façade design, its optimization and fabrication plan. In these sections we explain both the concept and the way the functions are implemented, and we illustrate them with examples of the possible parametric variations. The optimization process narrowed the design space towards a set of final design solutions, which are them detailed in the fabrication section.

__Running the Notebook__

This Algorithmic Design (AD) program was written in the Julia programming language and using the Khepri AD tool. Please install the requited dependencies (used packaged listed below) in order to run the notebook locally on your PC.

<img src="./figures/variations.png" width="900">

# Package Setup

__Install__:

In [ ]:
# using Pkg

In [ ]:
# CSV
# Pkg.add(PackageSpec(url="https://github.com/JuliaData/CSV.jl#01e46b4b264b2c73ed51f18282d128b57b76d097"))

In [ ]:
# DataFrames
# Pkg.add(PackageSpec(url="https://github.com/JuliaData/DataFrames.jl#473260902f36ee9fb1ede5b1c69b81255eb4e5b1"))

In [ ]:
# WebIO
# Pkg.add(PackageSpec(url="https://github.com/JuliaGizmos/WebIO.jl#de95ec71ac81484f50ccd1db68d96f55380f8b6f"))

In [ ]:
#PlotlyJS
# Pkg.add(PackageSpec(url="https://github.com/sglyon/PlotlyJS.jl#6580f9868ae8ada802eca8dc1327311b98436982"))

In [ ]:
# Interact
# Pkg.add(PackageSpec(url="https://github.com/JuliaGizmos/Interact.jl#dd595cfd24feaaebba8c5456d5b7760f239b4241"))

In [ ]:
# OpenStreetMap
# Pkg.add(PackageSpec(url="https://github.com/pszufe/OpenStreetMapX.jl"))

In [ ]:
# Khepri
# Pkg.add(PackageSpec(url="https://github.com/aptmcl/Khepri.jl"))

In [ ]:
# Pkg.update("Khepri")

In [ ]:
# ADOPT
# Pkg.add(PackageSpec(url="https://github.com/PastelBelem8/ADOPT.jl#5496aeda232873d0e908b1714d789847d1d4353a")

__Use:__

Site:

In [1]:
using OpenStreetMapX

ArgumentError: ArgumentError: Package OpenStreetMapX not found in current path:
- Run `import Pkg; Pkg.add("OpenStreetMapX")` to install the OpenStreetMapX package.


AD:

In [2]:
using Base.Iterators

In [3]:
using Khepri

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-7646894253024058357\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-5716622655110525918\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-7646894253024058357\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

Visualizers:

In [4]:
using Logging

In [5]:
using WebIO

In [6]:
using PlotlyJS

In [ ]:
using Interact

Optimization:

In [ ]:
using CSV

In [ ]:
using DataFrames

In [ ]:
# using ADOPT

In [ ]:
# using ADOPT.Sampling

In [ ]:
# using ADOPT.Platypus

## Notebook Macros

__Outlining__

The following macro allows you to use outlining in this notebook. `avoid_tests`, when set to true, will allow you to run the run the entire notebook skipping intermediate tests. When set to false, you may run the test spread trough the document, which illustrate the program's functions.

In [ ]:
avoid_tests = Khepri.Parameter(true)

macro test(expr...)
  quote
    if !avoid_tests() 
        begin
            $(esc(expr...))
        end
    end
  end
end  

In [ ]:
# avoid_tests(false)

__Remove logs:__ avoid log messages as cell outputs.

In [ ]:
Logging.disable_logging(Logging.Info)

# Site Map

We aquired the site's information from OpenStreetMap, namely the urban surrounding plan and building height information. 

The location is Av. República n7, Lisbon, Portugal.

Get .osm map file from: https://www.openstreetmap.org/way/96884765

| Open Street Map Site Plan                         |  Google Street View of Building Site              |
|---------------------------------------------------|---------------------------------------------------|
| <img src="./figures/open_street_map.png" width="500"> | <img src="./figures/google_street_view.png" width="500"> |

| Google globe view                         |
|-------------------------------------------|
| <img src="./figures/google_globe_bb.png" width="1000"> | 

Functions that generate the buildings 3D model using the exported information (plan vertices and height):

`building_height` finds the building height from whatever information is available in the extracted map.

In [ ]:
building_height(way) =
  begin
    for key in ["height", "building:height", "min_height", "roof:height"]
      if haskey(way.tags, key)
        return parse_length(way.tags[key])
      end
    end
    for key in ["building:levels", "roof:levels"]
      if haskey(way.tags, key)
        return parse_length(way.tags[key])*3
      end
    end
    30 # default building height
  end

`parse_length` returns the length value in meters.

In [ ]:
parse_length(str) =
  if endswith(str, "m")
    parse(Float64, str[1:end-1])
  elseif endswith(str, "ft")
    parse(Float64, str[1:end-2])*0.3048 # feet to meter conversion
  else
    parse(Float64, str)
  end

`building_data` returns the building outline (point array) and its height.

In [ ]:
building_data(nodes, way) =
  let waynodes = way.nodes,
      height = building_height(way),
      enus = [nodes[id] for id in waynodes],
      pts = [xyz(l.east, l.north, l.up) for l in enus],
      z = cz(pts[1])
    ([xyz(pt.x, pt.y, z) for pt in pts], height)
  end

`is_building` tests if an open street map element is a building.

In [ ]:
is_building(way) =
  haskey(way.tags, "building")

`city_data` returns the building data of all building elements on the map.

In [ ]:
city_data(map) =
  let nodes = OpenStreetMapX.ENU(map.nodes, first(values(map.nodes)))
    [building_data(nodes, way)
     for way in map.ways
     if is_building(way)]
  end

Given a file name for an OpenStreetMap file, `buildings_from_osm` generates the 3D model of the site:

In [ ]:
buildings_data_from_osm(filename) =
  let m = OpenStreetMapX.parseOSM(joinpath(@__DIR__, filename))
    city_data(m)
  end

osm_building((pts, h)) = irregular_prism(pts[1:end-1], h)

buildings_from_osm(filename) =
  osm_building.(buildings_data_from_osm(filename))

Generate original site 3D model of urban surroundings:

In [ ]:
@test begin
    backend(meshcat)
#     backend(autocad)
#     backend(rhino)
#     backend(revit)
    render_size(900,400)
end

In [ ]:
@test begin
    new_backend()
end;

The test should show the site with the project building included in the surroundings:

In [ ]:
@test begin
    delete_all_shapes()
    buildings_from_osm("map.osm")
    cylinder(u0(), 0.1, 1.8) # person attempt
end

Expected result (meshcat):

<img src="./figures/site_3D_meshcat.png" width="800">

| Plan site in AutoCAD                       |  3D site model in AutoCAD             |
|---------------------------------------------------|---------------------------------------------------|
| <img src="./figures/site_autocad_plan_color.png" width="390"> | <img src="./figures/site_autocad_3D_color.png" width="630"> |

| 3D site model in Rhino             |
|-------------------------------------------|
| <img src="./figures/location_rhino.png" width="1000"> | 

The test should show the site without the project building:

In [ ]:
@test begin
    delete_all_shapes()
    osm_building.(buildings_data_from_osm("map.osm")[1:end .!= 188])
end

| 3D site model in Revit without project building [188]             |
|-------------------------------------------|
| <img src="./figures/surroundings_revit.png" width="1000"> | 

The test should show the project building only:

In [ ]:
@test begin
    delete_all_shapes()
    osm_building(buildings_data_from_osm("map.osm")[188])
end

Expected result (meshcat):
<img src="./figures/project_build_mesh.png" width="300">

# Facade Concept

__Tiling__

A tiling of the plane is a family of sets - called tiles - that cover the plane without gaps or overlaps. Tiling is also known as tessellation, paving, or mosaics; they have appeared in human activities since prehistoric time, including architecture. This technique has found several applications in architecture, namely, to explore both its tectonic and visual expressiveness to generate building skins with multiple geometric patterns and performance behaviors (e.g., shading, ventilation, lighting proprieties, among others). 

In this particular design, we use tiling to create a visually interesting/dynamic geometric pattern, as well as to shade the interior spaces of the building: we opted for using irregular triangular tiles, which we materialize as triangular-shaped elements that can have different sizes and opacity levels.

| Tile aperture shapes and sizes |
|-------------------------------------------------------------------------------------|
| <img src="./figures/tiles_paint.png" width="400">|

__On site__

The application of the pattern to the building's façade intends to both create a visually interesting/dynamic geometric effect, and to shade the interior spaces of the building from the intense natural daylight typical of southern European countries. The pattern creates an interesting light effect on the inside office spaces.

| Hand-made drawing of the initial design intent for the location |
|---------------------------------------------------|
| <img src="./figures/sketch_facade1.png" width="700">|

#  Algorithmic Description

## Auxiliar Functions

This section contains auxiliary functions for the development of the façade tiling design

This function creates a constant value.

In [ ]:
always(x) = i -> x

| Polygon center        |
|---------------------------------------------------|
| <img src="./figures/polygon_center.png" width="400"> |

The function `polygon_center` receives the vertices defining a polygon, and returns its center. 

In [ ]:
polygon_center(pts) = 
    let n = length(pts),
        xs = [cx(p) for p in pts],
        ys = [cy(p) for p in pts],
        zs = [cz(p) for p in pts]
        xyz(sum(xs)/n, sum(ys)/n, sum(zs)/n)
    end

## Distributions

| Tile pattern distribution options: conceptual sketch        |
|---------------------------------------------------|
| <img src="./figures/sketch1_distributions.jpg" width="800"> |

In [ ]:
@test begin
    backend(notebook)
    render_size(800, 400)
end

The function `itera_squares` receives a matrix of points defining a surface and iterates over this matrix, rearranging those points in a regular triangular grid.

In [ ]:
itera_squares(ptss) =
    vcat([[[p0, p1, p2, p3]
           for (p0, p1, p2, p3) in zip(pts0[1:end-1], pts1[1:end-1], pts1[2:end], pts0[2:end])]
          for (pts0, pts1) in zip(ptss[1:end-1], ptss[2:end])]...)

The function `transpose_matrix` converts the matrix columns to rows and rows to columns.

In [ ]:
transpose_matrix(matrix) =
    [[row[i] for row in matrix]
    for i in 1:length(matrix[1])]

In [ ]:
@test begin
    new_backend()
    ptss = map_division(xz, 0, 10, 10, 0, 10, 10)
    map(Khepri.line, ptss)
    map(Khepri.line, transpose_matrix(ptss))
end;

Expected result:
<img src="./figures/distribution_matrix.png" width="400">

In [ ]:
my_polygon(pts) = Khepri.line(pts..., pts[1])

In [ ]:
@test begin
    new_backend()
    ptss = map_division(xz, 0, 5, 5, 0, 5, 5)
    map(my_polygon, itera_squares(ptss))
end;

Expected result:
<img src="./figures/distribution_squares.png" width="400">

The function `itera_2triangs` receives a matrix of points defining a surface and rearranges those points in a triangular grid.

In [ ]:
itera_2triangs(ptss) =
    vcat([vcat([[[p0,p1,p3],[p1,p2,p3]]
                for (p0,p1,p2,p3)
                in zip(pts0[1:end-1], pts1[1:end-1], pts1[2:end], pts0[2:end])]...)
         for (pts0, pts1) in zip(ptss[1:end-1], ptss[2:end])]...)

In [ ]:
@test begin
    new_backend()
    ptss = map_division(xz, 0, 3, 3, 0, 3, 3)
    map(my_polygon, itera_2triangs(ptss))
end;

Expected result:
<img src="./figures/distribution_triang.png" width="400">

The function `itera_2triangs_mirrorXY` receives a matrix of points defining a surface and rearranges those points in an irregular triangular grid: instead of being equally placed, the triangles mirror themselves in both _u_ and _v_ dimensions.

In [ ]:
itera_2triangs_mirrorXY(ptss) =
    vcat([vcat([if is_on
                   is_on = !(is_on)
                   [[p0,p1,p3],[p1,p2,p3]]
               else
                   is_on = !(is_on)
                   [[p0,p1,p2],[p2,p3,p0]]
               end
               for (p0,p1,p2,p3)
               in zip(pts0[1:end-1], pts1[1:end-1], pts1[2:end], pts0[2:end])]...)
          for (pts0,pts1,is_on)
          in zip(ptss[1:end-1], ptss[2:end], cycle([true, false]))]...)

In [ ]:
@test begin
    new_backend()
    ptss = map_division(xz, 0, 3, 3, 0, 3, 3)
    map(my_polygon, itera_2triangs_mirrorXY(ptss))
end;

Expected result:
<img src="./figures/distribution_mirror.png" width="400">

## Pattern

| Tile pattern conceptual sketch: left - design variables, right - design variations   |
|---------------------------------------------------|
| <img src="./figures/sketch1_concept.jpg" width="800"> |

| Hand-made drawing of the tiling facade with triangular tiles with varying opacity levels (first distribution option explored) |
|---------------------------------------------------|
| <img src="./figures/sketch_facade2.png" width="700"> |

__Tile design concept:__
* Façade composed of polygonal tiles of different opacities.
* The shape of the tiles varies according to the façade surface grid: a squared grid creates squared tiles, a triangular grid creates triangular tiles, etc.
* The opacity of the tiles should vary according to daylight we wish to let into the interior space.

The function `polygonal_tile` creates one tile whose opacity can vary between 0% (totally opaque) and 100% (totally transparent).

__Parameters:__ Polygon vertices and aperture factor.

* `pts`= two-dimensional distribution of points: matrix of points instead of an array of points
* `factor`= aperture factor of the tiles
* the function returns tiles' whose shape is automatically calculated based on the points distribution and the factor given.

In [ ]:
polygonal_tile(pts) =
    (; factor=0.5) ->
    let p = polygon_center(pts)
        qts = [intermediate_loc(p,q,factor) for q in pts]
        my_polygon(pts)
        my_polygon(qts)
    end

In [ ]:
@test begin
    backend(notebook)
    render_size(600, 400)
end

In [ ]:
@test begin
    new_backend()
    @manipulate for f=widget(0.1:0.1:0.9, label="Aperture factor")
                delete_all_shapes()
                polygonal_tile([x(0), x(1), xz(1,1), z(1)])(factor=f)
                nothing
    end
end

Expected result:
<img src="./figures/tile_lines.png" width="900">

__3D object tile (surface)__

The function `polygonal_tile` redefined to create 3D CAD elements (surface polygons)

In [ ]:
polygonal_tile(pts) =
    (; factor=0.5) ->
    let p = polygon_center(pts)
        qts = [intermediate_loc(p,q,factor) for q in pts]
      subtraction(surface_polygon(pts), surface_polygon(qts))
    end

In [ ]:
@test begin
    backend(rhino)
    render_size(1000, 1000)
end

In [ ]:
@test begin
    polygonal_tile([x(0), x(1), xz(1,1), z(1)])(factor=0.5)
end;

Expected result:
<img src="./figures/pattern_concept.png" width="300">

__Façade pattern: apply to the tiles__

The function `pattern` receives the pattern element and creates an entire facade pattern.

__Parameters:__ Pattern conceptual element, surface points, and _n_ additional parameters of the pattern conceptual element.

* `pts`= two-dimensional distribution of points: matrix of points instead of an array of points
* `fshape` = function as an argument (e.g use `polygonal_tile` as input, with respective parameters)
* the output of the function is a surface pattern

In [ ]:
pattern(fshape, pts...; args...) =
  fshape(pts...)(; (k=>v(pts) for (k,v) in args)...)

### Squared pattern

In [ ]:
@test begin
    backend(rhino)
    render_size(1000, 1000)
end

Simple __regular pattern__ using squared tiles.

In [ ]:
@test begin
    ptss = map_division(xz, 0, 10, 10, 0, 10, 10)
    pattern.(polygonal_tile,
             itera_squares(ptss),
             factor=always(0.5))
end

Expected result:
<img src="./figures/test1.png" width="500">

Simple __random pattern__ using squared tiles. 
The aperture factor for each tile is defined by a `random_range`.

In [ ]:
@test begin
    delete_all_shapes()
    ptss = map_division(xz, 0, 10, 10, 0, 10, 10)
    pattern.(polygonal_tile,
         itera_squares(ptss),
         factor=pts->random_range(0.1,1.0))
end

Expected result:
<img src="./figures/test2.png" width="500">

### Triangular pattern

In [ ]:
@test begin
    backend(rhino)
    render_size(1000, 1000)
end

Simple __random pattern__ using regular triangular tiles.

In [ ]:
@test begin
    delete_all_shapes()
    ptss = map_division(xz, 0, 10, 10, 0, 10, 10)
    pattern.(polygonal_tile,
         itera_2triangs(ptss),
         factor=pts->random_range(0.1,1.0))
end

Expected result:
<img src="./figures/test3.png" width="500">

### Mirrored triangular tiles

In [ ]:
@test begin
    backend(rhino)
    render_size(1000, 1000)
end

More complex __random pattern__ using mirrored triangular tiles.

In [ ]:
@test begin
    delete_all_shapes()
    ptss = map_division(xz, 0, 10, 10, 0, 10, 10)
    pattern.(polygonal_tile,
         itera_2triangs_mirrorXY(ptss),
         factor=pts->random_range(0.1,1.0))
end

Expected result:
<img src="./figures/test4.png" width="500">

Design variations __controlling the randomness__ of the tiles' aperture factor:

Controlled randomness: __central horizontal stain__.

* Apperture factor is influenced by a sinusoidal wave, whose movement depends on the `x` coordinate of the corner of each tile.

$$
F_{aperture}(p) = sin \left( \frac{cx \left(p \right)}{10} \times \pi \right) - 0.1
$$

In [ ]:
@test begin
    delete_all_shapes()
    ptss = map_division(xz, 0, 10, 10, 0, 10, 10)
    pattern.(polygonal_tile,
         itera_2triangs_mirrorXY(ptss),
         factor=pts->sin(pts[1][1].x/10*pi)-0.1)
end

Expected result:
<img src="./figures/test5.png" width="500">

Controlled randomness: __central stain__.

* Aperture factor is influenced by a sinusoidal wave, whose movement depends on the `x` and `z` coordinates of the corner of each tile.

$$
F_{aperture}(p) = sin \left( \frac{cx \left(p \right)}{10} \times \pi \right) \times sin \left( \frac{cz \left(p \right)}{10} \times \pi \right) - 0.1
$$

In [ ]:
@test begin
    delete_all_shapes()
    ptss = map_division(xz, 0, 10, 10, 0, 10, 10)
    pattern.(polygonal_tile,
         itera_2triangs_mirrorXY(ptss),
         factor=pts->sin(pts[1][1].x/10*pi)*sin(pts[1][1].z/10*pi)-0.1)
end

Expected result:
<img src="./figures/test6.png" width="500">

Controlled randomness: __horizontal increasing stain__.

* Apperture factor is influenced by a sinusoidal wave, whose movement depends on the `x` coordinate of the corner of each tile.

$$
F_{aperture}(p) = sin \left( \frac{cx \left(p \right)}{10} \times \frac{\pi}{2} \right) - 0.1
$$

In [ ]:
@test begin
    delete_all_shapes()
    ptss = map_division(xz, 0, 10, 10, 0, 10, 10)
    pattern.(polygonal_tile,
         itera_2triangs_mirrorXY(ptss),
         factor=pts->sin(pts[1][1].x/10*(pi/2))-0.1)
end

Expected result:
<img src="./figures/test7.png" width="500">

### Standard building box test

Definition of the generic building box elements dimensions (m):

In [ ]:
column_width = 0.3
wall_thickness = 0.3
glass_wall_thickness = 0.01
ext_glass_distance = 0.55

Definition of the building elements characteristics/family:

In [ ]:
top_slab_fam = slab_family_element(default_slab_family(), thickness=0.30)
bottom_slab_fam = slab_family_element(default_slab_family(), thickness=0.01)
wall_fam = wall_family_element(default_wall_family(), thickness=wall_thickness)
glass_wall_fam = wall_family_element(default_wall_family(), thickness=glass_wall_thickness)
column_fam = Khepri.column_family_element(default_column_family(), profile=rectangular_path(u0(), column_width, column_width));

Function that creates the building box: a simplification of the building geometry containing walls, slabs, and columns.

__Parameters:__ Position, building height, length, and width.
* Receives the (corner) position where to create the building box
* Receives three dimensions defining the volume of the building box
* Creates a 3D model composed of two slabs (ground and roof), four walls, and a structural column

In [ ]:
building_box(p0, height, length, width)=
    let
        p1=p0+vx(wall_thickness/2)
        p2=p1+vy(width)
        p3=p2+vxy(wall_thickness/2, -wall_thickness/2)
        p4=p3+vx(length-wall_thickness)
        p5=p1+vxy(wall_thickness/2)
        p6=p5+vx(length-wall_thickness-ext_glass_distance)
        p7=p4+vxy(-ext_glass_distance, -wall_thickness/2)
    
        slab_layer=create_layer("slab")
        wall_layer=create_layer("wall")
        glass_layer=create_layer("glass")

        Khepri.with(current_layer, slab_layer) do
            slab(rectangular_path(p0, length, width), family=bottom_slab_fam)
            slab(rectangular_path(p0+vz(height), length, width), family=top_slab_fam)
        end
        Khepri.with(current_layer, wall_layer) do
            wall([p1, p2], top_level=height-wall_thickness, family=wall_fam)
            wall([p3, p4], top_level=height-wall_thickness, family=wall_fam)
            column(p0+vx(length-column_width), top_level=height, family=column_fam)
        end
        Khepri.with(current_layer, glass_layer) do
            wall([p5, p6], top_level=height-wall_thickness, family=glass_wall_fam)
            wall([p7, p6], top_level=height-wall_thickness, family=glass_wall_fam)
        end
    end

Definition of generic south and east facades dimensions:

In [ ]:
f_south_length = 13.05
f_south_height = 3
f_east_length = 9.85 + 0.3
f_east_height = f_south_height

In [ ]:
function facade_test(min1=0.0, max1=0.9; n1=43, m=13, n2=32, opaqueness_factor=0.5)
    facade_layer=create_layer("facade")
    set_random_seed(1234)
    Khepri.with(current_layer, facade_layer) do
        Khepri.with(current_cs, cs_from_o_vx_vy(x(-0.30), vx(1), vz(1))) do
            s1= itera_2triangs_mirrorXY(map_division(xy, 0, f_south_length, n1, 0.3, f_south_height+0.3, m))
            pattern.(polygonal_tile,
                     s1,
                     factor=pts->random_range(min1, sin(pts[1][1].x/f_south_length*pi)-(1-max1)))
        end
        Khepri.with(current_cs, cs_from_o_vx_vy(x(f_south_length-0.3), vy(1), vz(1))) do
            s2 = map_division(xy, 0, f_east_length, n2, 0.3, f_east_height+0.3, m)
            surface_grid(s2)
      end
    end
end

### Vertical stain pattern distribution

__First pattern distribution choice__

We first settled on the central vertical stain distribution option, which offers the central area of the facade more sunlight exposure. The aperture factor of the tiles increases from the outskirts of the façade to the middle, influenced by a half sinusoid cycle. The resulting effect leaves the service areas (located on the outmost corners of the building) with less natural light, hence easier to control thermally during daylight hours. The central façade area is reserved for office spaces who sacrifice better thermal control for better daylight illumination during working hours.

| Plate engravings showing the tile grid on standard façade       |
|---------------------------------------------------|
| <img src="./figures/facade_concept.png" width="800">|

| No engravings option hides the tile grid that guided the perforation pattern |
|---------------------------------------------------|
| <img src="./figures/facade_concept_no_engravings.png" width="800">|

The function `facade`creates a simple building box with both east and south facades. The soth façade shows the tile pattern whereas the east one has a simple surface only.

In [ ]:
standard_box() = building_box(u0()+vxyz(-0.30, 0, 0.30), f_south_height, f_south_length, f_east_length)

In [ ]:
@test begin
    backend(rhino)
end

In [ ]:
@test begin
    delete_all_shapes()
    standard_box()
    facade_test()
end

Expected result:

<img src="./figures/facade_test1.png" width="600">

Experimenting with bigger openings:

In [ ]:
@test begin
    delete_all_shapes()
    standard_box()
    facade_test(0.3)
end

Expected result:

<img src="./figures/facade_test2.png" width="600">

Experimenting with even bigger openings:

In [ ]:
@test begin
    delete_all_shapes()
    standard_box()
    facade_test(0.5,0.95)
end

Expected result:

<img src="./figures/facade_test3.png" width="600">

## Placing the tiled façade on site

This section presented the definition of the building elements, namely the pre-existing geometry before the façade intervention (building box) and the façade intervention:

In [ ]:
# Building dimentions
n_floors = 11
int_wall_thickness = 0.15
slab_thickness = 0.3
floor_height = 3
default_level_to_level_height(floor_height + slab_thickness)

# Number of rooms per floor
n_rooms = 5

Automatically extract building corners from OpenStreetMap outline:

In [ ]:
building_path_pts = circshift(buildings_data_from_osm("map.osm")[188][1][1:end-1], 1)

In [ ]:
# Pre-existing lot contour path (site plan)
# openStreetMap manual correction of corners

building_path = closed_polygonal_path(
    [xyz(-17.502854,130.41505,0),
     xyz(-23.450086,159.78842,0),
     xyz(-40.379042,156.41009,0),
     xyz(-38.727039,148.99827,0),
     xyz(-48.752260,146.90023,0),
     xyz(-46.187342,133.90089,0),
     xyz(-36.388175,135.52159,0),
     xyz(-34.910079,126.94048,0)])

### BIM families

This section contains the redefinition of the BIM families we plan to change from the defaults in Khepri, along with the chosen materials for each backend.

In [ ]:
int_wall_fam = wall_family_element(default_wall_family(), thickness=int_wall_thickness)
slab_fam = slab_family_element(default_slab_family(), thickness=slab_thickness)
roof_fam = roof_family_element(default_roof_family(), thickness=0.20)
window_fam(l) = window_family_element(default_window_family(), width=l-0.2, height=default_level_to_level_height()-0.5)

tile_fam=panel_family_element(default_panel_family());

Materials for __Unity__ backend:

In [ ]:
set_backend_family(tile_fam, unity, unity_material_family("Default/Materials/White"))

Materials for __Meshcat__ backend:

In [ ]:
set_backend_family(tile_fam, meshcat, meshcat_material_family(meshcat_material(RGB(1,1,1))))
set_backend_family(roof_fam, meshcat, meshcat_material_family(meshcat_material(RGB(0.8,0.8,0.8))))

Materials for __POV-Ray__ backend:

In [ ]:
default_povray_material(povray_definition("Material", "texture",
    "{ pigment { color rgb 0.2 } finish { reflection 0 ambient 0 }}"))
set_backend_family(tile_fam, povray, povray_material_family(povray_material("Tile", gray=0.8)))
set_backend_family(int_wall_fam, povray, povray_wall_family(povray_material("IntWall", gray=0.8)))
set_backend_family(slab_fam, povray, povray_slab_family(povray_material("Slab", gray=0.8)))
set_backend_family(roof_fam, povray, povray_roof_family(povray_material("Roof", gray=0.9)))
#set_backend_family(window_fam, povray, povray_material_family(povray_material("GenericCeiling80", gray=0.5))

### BIM operations

The function `polygonal_tile` redefined to create 3D BIM elements (panel object) + in statement for minimum size openings:

In [ ]:
polygonal_tile(pts) =
    (; factor=0.5) ->
    let p = polygon_center(pts)
        qts = [intermediate_loc(p,q,factor) for q in pts]
        panel(pts, family=tile_fam, openings= Khepri.distance(qts[1], p) < 0.05 ? [] : [qts])
    end

In [ ]:
@test begin
    backend(meshcat)
    render_size(800, 400)
end

In [ ]:
@test begin
    new_backend()
    sleep(5)
    @manipulate for f=widget(0.3:0.1:0.9, label="Aperture factor")
                delete_all_shapes()
                polygonal_tile([x(0), x(1), xy(1,1), y(1)])(factor=f)
                nothing
    end
end

Expected result:
<img src="./figures/tile_3D.png" width="900">

New version of function `facade`, with parameters as input, and only generating a front facing façade for the pre-existing building:

* Aperture factor is influenced by a sinusoidal wave, whose movement depends on the `x` coordinate of the corner of each tile.

$$
F_{aperture}(p) = random_{range} \left \{ \begin{matrix} min_1 \\
                    sin \left( \frac{cx \left(p \right)}{f_{length}} \times \pi \right) - (1 - max_1)
                    \end{matrix} \right.
$$

In [ ]:
function facade_central_stain_tiles(origin, direction, length, height, min1=0.15, max1=0.9; n1=43, m=13)
    facade_layer = create_layer("Facade_Tiles")
    set_random_seed(1234)
    Khepri.with(current_layer, facade_layer) do
        Khepri.with(current_cs, cs_from_o_vx_vy(origin+vx(0.1), direction, vz(1))) do
            s1= itera_2triangs_mirrorXY(map_division(xy, 0, length, n1, 0, height, m))
            pattern.(polygonal_tile,
                     s1,
                     factor=pts->random_range(min1, max(min1, sin(pts[1][1].x/length*π)-(1-max1))))
        end
    end
end

In [ ]:
@test begin
    backend(autocad)
    delete_all_shapes()
    pts = path_vertices(building_path)
    pt1, pt2, pt3, pte = pts[[1, 2, 3, end]]
    min1=0.15
    max1=0.9
    n1=43
    m=13
    facade_central_stain_tiles(pt1, pt2-pt1, 13, 3, min1, max1, n1=n1, m=m)
    set_view(xyz(5.07,137.275,1.041), xyz(-18.692,137.275,1.041), 50.0)
end

In [ ]:
test_facade_central_stain_tiles() = 
 begin
    backend(autocad)
    delete_all_shapes()
    pts = path_vertices(building_path)
    pt1, pt2, pt3, pte = pts[[1, 2, 3, end]]
    min1=0.15
    max1=0.9
    n1=43
    m=13
    facade_central_stain_tiles(pt1, pt2-pt1, 13, 3, min1, max1, n1=n1, m=m)
    set_view(xyz(5.07,137.275,1.041), xyz(-18.692,137.275,1.041), 50.0)
end

In [ ]:
@test begin
    test_facade_central_stain_tiles()
end

Expected result:
<img src="./figures/distribution1.png" width="900">

### Refactoring Façade Tiles Function

As we are going to tests several design variants, we now generalize the `facade_tiles` function.

In [ ]:
function facade_tiles(factor, origin, direction, length, height; n1=43, m=13)
    facade_layer = create_layer("Facade_Tiles")
    set_random_seed(1234)
    Khepri.with(current_layer, facade_layer) do
        Khepri.with(current_cs, cs_from_o_vx_vy(origin+vx(0.1), direction, vz(1))) do
            s1= itera_2triangs_mirrorXY(map_division(xy, 0, length, n1, 0, height, m))
            pattern.(polygonal_tile,
                     s1,
                     factor=factor)
        end
    end
end

In [ ]:
test_facade_tiles(factor) = 
 begin
    backend(autocad)
    delete_all_shapes()
    pts = path_vertices(building_path)
    pt1, pt2, pt3, pte = pts[[1, 2, 3, end]]
    min1=0.15
    max1=0.9
    n1=43
    m=13
    facade_tiles(factor(min1, max1, m), pt1, pt2-pt1, 13, 3, min1, max1, n1=n1, m=m)
    set_view(xyz(5.07,137.275,1.041), xyz(-18.692,137.275,1.041), 50.0)
end

In [ ]:
@test begin
    test_facade_tiles((min1,max1,length)->pts->random_range(min1, max(min1, sin(pts[1][1].x/length*π)-(1-max1))))
end

Expected result (refactoring process should yield the same result as previous function):
<img src="./figures/distribution1.png" width="900">

### Multiple stains pattern distribution

| Hand-made drawing of the second pattern distribution option: multiple vertical stains |
|---------------------------------------------------|
| <img src="./figures/new_distribution_sketch.jpg" width="700"> |

__Second pattern distribution choice__

After some preliminary analysis we concluded the light coming into the service spaces and corner offices was not enough for our standard metrics and we changed the distribution to accommodate the same design effect but several times along the façade's width. Specifically, we indented the sinusoidal wavelength to match the number of large office division inside, so that each big room could appreciate the fading effect from the inside in its entirety.

_old mathematical expression for aperture pattern:_

$$
F_{aperture}(p) = random_{range} \left \{ \begin{matrix} min_1 \\
                    sin \left( \frac{cx \left(p \right)}{f_{length}} \times \pi \right) - (1 - max_1)
                    \end{matrix} \right.
$$

In [ ]:
@test begin
    test_facade_tiles((min1,max1,length)->
        pts->random_range(min1, max(min1, sin(pts[1][1].x/length*π)-(1-max1))))
end

Expected result:
<img src="./figures/distribution1.png" width="900">

_new mathematical expression for aperture pattern:_

$$
F_{aperture}(p) = random_{range} \left \{ \begin{matrix} min_1 \\
                    \left | sin \left( \frac{cx \left(p \right)}{f_{length}} \times n_{rooms} \times \pi \right) \right | - (1 - max_1)
                    \end{matrix} \right.
$$

In [ ]:
@test begin
    test_facade_tiles((min1,max1,length)->
        pts->random_range(min1, max(min1, (abs ∘ sin)(pts[1][1].x/length*n_rooms*π)-(1-max1))))
end

Expected result:
<img src="./figures/distribution2.png" width="900">

## Complete building

The `building` function generates the entire building lot with the new façade in place:

In [ ]:
building(slab_path=building_path, min1=0.15, max1=0.9; n1=43, m=13) =
    let pts = path_vertices(slab_path) # Building plan contour path vertices
        pt1, pt2, pt3, pte = pts[[1, 2, 3, end]]
        b1 = intermediate_loc(pt1, pte, 0.5)
        b2 = intermediate_loc(pt2, pt3, 0.5)
        f_length = Khepri.distance(pts[1:2]...)
        f_height = default_level_to_level_height() * n_floors + roof_fam.thickness
        default_level(0)
        for i in 1:n_floors
            slab(slab_path, family=slab_fam)
            add_window(wall(slab_path), xy(0.1, 0.1), window_fam(f_length))
            wall([b1, b2], family=int_wall_fam)
            for ϵ in division(0, 1, n_rooms)[2:end-1]
                wall([intermediate_loc(b1, b2, ϵ), intermediate_loc(pt1, pt2, ϵ)], family=int_wall_fam)
            end
            default_level(upper_level())
        end
        roof(slab_path, family=roof_fam)
        facade_tiles(pt1, pt2-pt1, f_length, f_height, min1, max1, n1=n1, m=m)
    end

In [ ]:
@test begin
    backend(autocad)
    delete_all_shapes()
    building()
end;

Expected result:

| Plan view in AutoCAD                                | Conceptual elevation in AutoCAD                 |
|-----------------------------------------------------|-------------------------------------------------|
| <img src="./figures/building_plan.png" width="400"> | <img src="./figures/building_m.png" width="400">|

Restricting `m`'s size to match that of `n`. This constrain forces the tiles to have a squared shape.

In [ ]:
building(slab_path=building_path, min1=0.15, max1=0.9; n1=43) =
    let pts = path_vertices(slab_path) 
        pt1, pt2, pt3, pte = pts[[1, 2, 3, end]]
        b1 = intermediate_loc(pt1, pte, 0.5)
        b2 = intermediate_loc(pt2, pt3, 0.5)
        f_length = Khepri.distance(pts[1:2]...)
        f_height = default_level_to_level_height() * n_floors + roof_fam.thickness
        n = f_length / n1
        m = floor(f_height / n)
        default_level(0)
        for i in 1:n_floors
            slab(slab_path, family=slab_fam) # slabs

            with_wall(slab_path) do w
              add_window(w, xy(0.1, 0.1), window_fam(f_length)) # contour walls + windows
            end

            wall([b1, b2], family=int_wall_fam) # interior back wall
            for ϵ in division(0, 1, n_rooms)[2:end-1] # interior office walls
                wall([intermediate_loc(b1, b2, ϵ), intermediate_loc(pt1, pt2, ϵ)], family=int_wall_fam)
            end
            default_level(upper_level())
        end
        roof(slab_path, family=roof_fam) # roof slab
        facade_tiles(pt1, pt2-pt1, f_length, f_height, min1, max1, n1=n1, m=m) # façade
    end

In [ ]:
@test begin
    backend(rhino)
    @manipulate for n_rooms_widget=widget(2:1:10, label="Number of rooms"),
                min_op=widget(0.1:0.05:0.85, label="Miminum opening"),
                max_op=widget(0.3:0.05:0.95, label="Maximum opening"),
                n=widget(2:1:50, label="Number of horizontal panels")
                n_rooms = n_rooms_widget[]
                delete_all_shapes()
                set_view(xyz(33.55,117.28,19.60), xyz(32.60,117.58,19.58), 30)
                building(building_path, min_op, max_op, n1=n)
                nothing
            end
end

Expected result:
<img src="./figures/building_interact.png" width="900">

In [ ]:
# restore n_rooms value
n_rooms = 5

In [ ]:
@test begin; backend(meshcat); render_size(1200, 1600); new_backend(); end;

In [ ]:
@test begin; delete_all_shapes(); building(); end;

Expected result:
<img src="./figures/meshcat_building.png" width="800">

In [ ]:
@test begin
    backend(autocad)
    delete_all_shapes()
    building()
end;

Expected result:

| Elevation in AutoCAD with visible tile edges        | Elevation in AutoCAD with no tile contour       |
|-----------------------------------------------------|-------------------------------------------------|
| <img src="./figures/building_n_m.png" width="400"> | <img src="./figures/building_n_m_realistic.png" width="400">|

In [ ]:
@test begin
    backend(rhino)
    delete_all_shapes()
    building()
end;

Expected result:

| Plate engravings showing the multiple stain tile grid in Rhino |
|--------------------------------------------------|
| <img src="./figures/with_lines.png" width="750"> | 

|  No engravings option hides the multiple stain tile grid in Rhino |
|---------------------------------------------------|
|<img src="./figures/without_lines.png" width="750">|

## Generate building + entire site

Site surroundings 3D buildings from OpenStreetMap:

In [ ]:
surroundings() = osm_building.(buildings_data_from_osm("map.osm")[1:end .!= 188])

In [ ]:
@test begin
    backend(meshcat)
    render_size(800, 400)
    new_backend()
    end;

In [ ]:
@test begin
    delete_all_shapes()
    current_layer(create_layer("surroundings", true, RGB(0.8,0.8,0.8))) # sourroundigns color
    surroundings()
    building()
    end;

Expected result:

<img src="./figures/meshcat_building_sur.png" width="800">

# Analysis

## Façade for Simulation

In [ ]:
sim_dir = joinpath(pwd(), "scripts")

__Trigger Simulation Functions__

In [ ]:
wait_and_read_file(results_path, tries=100, wait=20) =
  if tries==0
    @error("Could not read the results file $(results_path)")
  else
    if ispath(results_path)
      sleep(1) # It might still be writing
      open(results_path) do f
        readlines(f)
      end
    else
      sleep(wait)
      wait_and_read_file(results_path, tries-1, wait)
    end
 end

In [ ]:
trigger_simulation(trigger_path, results_path) =
  begin
    rm(results_path, force=true)
    touch(trigger_path)
    wait_and_read_file(results_path)
  end

__Prepare for Simulation__

For the simulation, we used Rhino in combination with a Grasshopper program using DIVA to calculate the desired daylight metrics, in our case ASE, sDA and sUDI.

`analysis_surface`: Function to select the desired room in a specific room.

In [ ]:
analysis_surface(; slab_path=building_path, floor_idx=5, room_idx=3) =
    let analysis_layer = create_layer("Floor")
        pts = path_vertices(slab_path) # Building plan contour path vertices
        pt1, pt2, pt3, pte = pts[[1, 2, 3, end]]
        b1 = intermediate_loc(pt1, pte, 0.5)
        b2 = intermediate_loc(pt2, pt3, 0.5)
        f_length = Khepri.distance(pts[1:2]...)
        f_height = default_level_to_level_height() * n_floors + roof_fam.thickness
        δ = 0.2
        ϵs = division(0, 1, n_rooms)[room_idx:room_idx+1]
        pa = intermediate_loc(b1, b2, ϵs[1])
        pb = intermediate_loc(pt1, pt2, ϵs[1])
        pc = intermediate_loc(pt1, pt2, ϵs[2])
        pd = intermediate_loc(b1, b2, ϵs[2])
        path = offset(translate(closed_polygonal_path([pa, pb, pc, pd]), vz((floor_idx-1)*default_level_to_level_height()+0.05)), δ)
        Khepri.with(current_layer, analysis_layer) do 
            fill(path)
        end
    end

`facade_tiles_vis`: Function that generates only the façade in the correct layer.

In [ ]:
facade_tiles_vis(slab_path=building_path; min1=0.0, max1=0.9, n1=43)=
    let facade_layer = create_layer("Facade_Tiles")
        pts = path_vertices(slab_path)
        pt1, pt2 = pts[[1, 2]]
        f_length = Khepri.distance(pts[1:2]...)
        n = f_length / n1
        f_height = default_level_to_level_height() * n_floors + roof_fam.thickness
        m = floor(f_height / n)
        delete_all_shapes_in_layer(facade_layer)
        facade_tiles(pt1, pt2-pt1, f_length, f_height, min1, max1, n1=n1, m=m)
    end

`facade_tiles_sim`: Function that generates the façade and then triggers the simulation for ASE, sDA and sUDI.

In [ ]:
facade_tiles_sim(slab_path=building_path; min1=0.0, max1=0.9, n1=43)=
    begin facade_tiles_vis(min1=min1, max1=max1, n1=n1)

        res = trigger_simulation(joinpath(sim_dir, "start.txt"),
                                 joinpath(sim_dir, "results.txt"))
        println(res)
        res[1] == "None" ? [100000.0, -100000.0, 1.0] : parse.(Float64, res)
    end

## Analysis Results

In [ ]:
@test begin
    backend(rhino)
end

For the analysis part we simulated the daylight each room of 3 different floors (2st, 7th and 11th) received.

The metric chosen to address this was the Useful Daylight Illuminance (UDI), and we presented the results for UDI 300-3000 lux, as well as the overlit and underlit UDI.

### Analysis 1

* __Min Aperture__ = 0.15 meters and __N__ = 40

| Corresponding façade.                              |
|----------------------------------------------------|
| <img src="./figures/sim01_facade.png" width="400"> |

In [ ]:
@test begin
    delete_all_shapes_in_layer("Floor")
    for i in 1:n_rooms
        analysis_surface(floor_idx=2, room_idx=i)
    end
    facade_tiles_sim(min1=0.15, n1=40)
end;

| 2nd Floor: Results for UDI 300-3000 lux             |
|-----------------------------------------------------|
| <img src="./figures/sim01_f01_UDI.png" width="750"> |


| 2nd Floor: Results for UDI 0-300 lux \| Underlit          | 2nd Floor: Results for UDI 3000 lux < \| Overlit         |
|-----------------------------------------------------------|----------------------------------------------------------|
| <img src="./figures/sim01_f01_UDI_under.png" width="500"> | <img src="./figures/sim01_f01_UDI_over.png" width="500"> |

In [ ]:
@test begin
    delete_all_shapes_in_layer("Floor")
    for i in 1:n_rooms
        analysis_surface(floor_idx=7, room_idx=i)
    end
    facade_tiles_sim(min1=0.15, n1=40)
end;

| 7th Floor: Results for UDI 300-3000 lux            |
|-----------------------------------------------------|
| <img src="./figures/sim01_f06_UDI.png" width="750"> |


| 7th Floor: Results for UDI 0-300 lux \| Underlit         | 7th Floor: Results for UDI 3000 lux < \| Overlit        |
|-----------------------------------------------------------|----------------------------------------------------------|
| <img src="./figures/sim01_f06_UDI_under.png" width="500"> | <img src="./figures/sim01_f06_UDI_over.png" width="500"> |

In [ ]:
@test begin
    delete_all_shapes_in_layer("Floor")
    for i in 1:n_rooms
        analysis_surface(floor_idx=n_floors, room_idx=i)
    end
    facade_tiles_sim(min1=0.15, n1=40)
end;

| 11th Floor: Results for UDI 300-3000 lux           |
|-----------------------------------------------------|
| <img src="./figures/sim01_f11_UDI.png" width="750"> |


| 11th Floor: Results for UDI 0-300 lux \| Underlit        | 11th Floor: Results for UDI 3000 lux < \| Overlit       |
|-----------------------------------------------------------|----------------------------------------------------------|
| <img src="./figures/sim01_f11_UDI_under.png" width="500"> | <img src="./figures/sim01_f11_UDI_over.png" width="500"> | 

### Analysis 2

* __Min Aperture__ = 0.40 meters and __N__ = 10

| Corresponding façade.                              |
|----------------------------------------------------|
| <img src="./figures/sim02_facade.png" width="400"> |

In [ ]:
@test begin
    delete_all_shapes_in_layer("Floor")
    for i in 1:n_rooms
        analysis_surface(floor_idx=2, room_idx=i)
    end
    facade_tiles_sim(min1=0.40, n1=10)
end;

| 2st Floor: Results for UDI 300-3000 lux            |
|-----------------------------------------------------|
| <img src="./figures/sim02_f01_UDI.png" width="750"> |


| 2st Floor: Results for UDI 0-300 lux \| Underlit         | 2st Floor: Results for UDI 3000 lux < \| Overlit        |
|-----------------------------------------------------------|----------------------------------------------------------|
| <img src="./figures/sim02_f01_UDI_under.png" width="500"> | <img src="./figures/sim01_f02_UDI_over.png" width="500"> |

In [ ]:
@test begin
    delete_all_shapes_in_layer("Floor")
    for i in 1:n_rooms
        analysis_surface(floor_idx=7, room_idx=i)
    end
    facade_tiles_sim(min1=0.40, n1=10)
end;

| 7th Floor: Results for UDI 300-3000 lux            |
|-----------------------------------------------------|
| <img src="./figures/sim02_f06_UDI.png" width="750"> |


| 7th Floor: Results for UDI 0-300 lux \| Underlit         | 7th Floor: Results for UDI 3000 lux < \| Overlit        |
|-----------------------------------------------------------|----------------------------------------------------------|
| <img src="./figures/sim02_f06_UDI_under.png" width="500"> | <img src="./figures/sim02_f06_UDI_over.png" width="500"> |

In [ ]:
@test begin
    delete_all_shapes_in_layer("Floor")
    for i in 1:n_rooms
        analysis_surface(floor_idx=n_floors, room_idx=i)
    end
    facade_tiles_sim(min1=0.40, n1=10)
end;

| 11th Floor: Results for UDI 300-3000 lux           |
|-----------------------------------------------------|
| <img src="./figures/sim02_f11_UDI.png" width="750"> |


| 11th Floor: Results for UDI 0-300 lux \| Underlit        | 11th Floor: Results for UDI 3000 lux < \| Overlit       |
|-----------------------------------------------------------|----------------------------------------------------------|
| <img src="./figures/sim02_f11_UDI_under.png" width="500"> | <img src="./figures/sim02_f11_UDI_over.png" width="500"> | 

# Optimization

Each one of the designed rooms has its own lighting conditions, meaning they should be analyzed individually to guarantee ideal conditions. However, for this work, we decided to optimize just one of them.

| Room selected for optimization.                      |
|------------------------------------------------------|
| <img src="./figures/selected_room.png" width="1000"> | 

The two metrics involved in the optimization process were Spatial Daylight Autonomy (sDA) and Annual Sun Exposure (ASE), and the objectives were:

$$
minimize \rightarrow f(x_1, x_2) = ASE_{1000,250h}(x_1, x_2)\\
maximize \rightarrow g(x_1, x_2) = sDA_{300|50\%} (x_1, x_2)
$$


x<sub>1</sub> and x<sub>2</sub> are the variables for our optimization problem, representing:
the min value for openings x<sub>1</sub> ∈ {0.1, 0.15, …, 0.85}, 
and the spacing between the panels x<sub>2</sub> ∈ {10, 11, …, 50}.


Finally, we tested the NSGA-II algorithm, with 250 solutions grouped in populations of 25.

## Optimization Process

### Initial Population

Generates the initial population for the metaheuristic algorithms, based on the Latin Hypercube algorithm.

In [ ]:
unscale(value, nmin, nmax, omin=0, omax=1) =
       (value .- omin) ./ (omax - omin) .* (nmax - nmin) .+ nmin

In [ ]:
get_initial_population(variables, pop_ini) = begin
    ADOPT.Sampling.set_seed!(12345)
    X = ADOPT.latinHypercube(length(variables), pop_ini)
    unscaled_X = zeros(size(X, 2), size(X, 1))

    for (i, var) in (enumerate(variables))
        unscaled_X[:, i] = unscale(X[i, :], ADOPT.lower_bound(var), ADOPT.upper_bound(var))
    end
    map(1:size(unscaled_X, 1)) do i
        round.(unscaled_X[i, :])
    end
end

### Optimization Objectives and Variables

__Objectives__

* The `map_to_int` function receives an integer and scales it to the a real number, based on a step. This allows us to treat the optimization problem as a discrete one, and to have more control over the variables range.
* `objective_ase` and `objective_sda` compute the ASE and sDA values, respectively.

In [ ]:
global_objs = -1

In [ ]:
map_to_int(x, min, step=0.01) = min + step*x

In [ ]:
objective_ase((min_opening, n)) = let
    min1 = map_to_int(min_opening, 0.1, 0.05)
    global global_objs
    global_objs=facade_tiles_sim(min1=min1, n1=n)
    global_objs[2]
end

In [ ]:
objective_sda(x)=global_objs[1]

In [ ]:
objs = [Objective(objective_ase, :MIN),
        Objective((x) -> -1 * objective_sda(x), :MIN)]

__Variables__

In [ ]:
min_opening = IntVariable(0, 15) # [0.1 , 0.85, step=0.05]
n = IntVariable(10, 50)

In [ ]:
vars = [min_opening, n]

### Optimization Setup

In [ ]:
res_dir = joinpath(pwd(), "algorithms")

__Metaheuristics__

Define the main parameters of the optimization:
* number of runs: `nruns`
* maximum number of evaluations: `maxevals`
* number of iterations: `niterations`
* population and initial population size: `nparticles` and `pop_ini`

In [ ]:
nruns = 1
maxevals = 200
niterations = 10
nparticles = div(maxevals, niterations)
pop_ini = nparticles

Define the optimization problem and the initial population.

In [ ]:
problem = Model(vars, objs)
initial_population = get_initial_population(vars, pop_ini)

Define the generator that will be used by the optimization algorithms to create the initial population.

In [ ]:
generator = Dict(:name => ADOPT.Platypus.InjectedPopulation,
                 :solutions => initial_population,
                 :problem => problem)

Define the parameters for the metaheuristic algorithms.

In [ ]:
EAs_params = Dict(:population_size => nparticles,
                  :generator => generator)

Start Simulation:

In [ ]:
# avoid_tests(false)

In [ ]:
@test begin
    backend(rhino)
end

In [ ]:
@test begin
    delete_all_shapes_in_layer("Floor")
    analysis_surface(floor_idx=8, room_idx=3)

    ADOPT.with(ADOPT.results_dir, res_dir) do
        benchmark(nruns=nruns,
        algorithms=[(NSGAII, EAs_params)],
        problem=problem, max_evals=maxevals)
    end
end

## Global Setup for Data Processing

Read the Julia file that contains the main functions for data processing and visualization.

In [ ]:
include("scripts/optimization_main_functions.jl")

Global constants for the data processing of the optimization results.

In [ ]:
# Folders Structure
base_folder = pwd()
results_folder = joinpath(base_folder, "algorithms")

# CSV File Configuration
has_header = true
files_sep = ","
file_extension = "csv"

# Optimization Settings
runs = [1]
nruns = length(runs)
max_evals = 250

## Problem Definition (in the files) 
### Variables
min_opening = :min_opening
n = :n
vars_cols = [min_opening, n]

### Objectives  
ASE = :ASE
sDA = :sDA
objs_cols = [ASE, sDA]

relevant_cols = vcat(vars_cols, objs_cols)

names_mapping = (
     4 => min_opening, 
     5 => n, 
     6 => ASE, 
     7 => sDA, 
)

## Multi-Objective Optimization Algorithms
### Metaheuristics
pop_size = 25
metaheuristics = ["NSGAII"]

### Model-Based (or metamodel)
metamodels_base = ["GPR"]
metamodels_strategies = ["SPEA2"]
metamodels_algorithms = ["$(b)_$(s)" for b in metamodels_base for s in metamodels_strategies]

all_algorithms = metaheuristics
# all_algorithms = vcat(metaheuristics, metamodels_algorithms)
n_algorithms = length(all_algorithms)

### Filenames with the results 
filenames = ["$(a)_results_0$(r).$(file_extension)" for r in runs for a in all_algorithms]
filenames

### Interactive Pareto Front Function

`create_pfs` creates an interactive Pareto front that, when clicking in point of plot generates the corresponding façade tiling, based on the `facade_tiles_vis` function

In [ ]:
function create_pfs(pfs; x, y, draw_dominated=true,
               names=all_algorithms, colorscale="viridis", colors=nothing,
               tpf=nothing, tpf_name="Combined_PF", tpf_color="rgb(0,0,0)",
               layout=layout)

    traces, custom_data = get_traces(pfs, x, y, draw_dominated, names, colorscale, colors, tpf, tpf_name, tpf_color, layout)

    fig = PlotlyJS.plot([traces...], layout)

    display(fig)
    on(fig.scope["click"]) do data
        let subdata = data["points"][1],
            point_idx = subdata["pointIndex"],
            curve_idx = subdata["curveNumber"],
            info = custom_data[curve_idx + 1][point_idx + 1, :],
            min_opening = info[:min_opening],
            n = info[:n]

          println(info)
          facade_tiles_vis(min1=min_opening, n1=n)
        end
    end
end

### Layout

Defines the layout in which the Pareto front plots will be displayed.

In [ ]:
layout_tiling = Layout(
    template="plotly_white",
    autosize=false,
    # Define plot size
    width=900, 
    height=540,
    # Legend Position
    # showlegend = False,
    legend=Dict(
        :orientation=>'h',
        :x=>-0.01,
        :y=>-0.2
    ),

    # Define axis
    xaxis=Dict(
        :title=>"sDA",
        :autorange=>true,
        :showgrid=>true,
        :zeroline=>false,
        :showline=>true,
        :ticks=>"",
        :showticklabels=>true,
        :tickformat=>"."
    ),
  
    yaxis=Dict(
        :title=>"ASE",
        :autorange=>true,        
        :showgrid=>true,
        :zeroline=>false,
        :showline=>true,
        :ticks=>"",
        :showticklabels=>true,
        :tickformat=>"."
    )
)

### Read Files

In [ ]:
# Read algorithms  
dfs = load_results(filenames);

In [ ]:
dfs = [rename!(df, [map(x->x[2], names_mapping) ...]) for df in dfs];

In [ ]:
# Compute non_dominated_solutions (per run)
pfs = [add_isdominated_cols(df) for df in dfs];

In [ ]:
# Computes combined Pareto Front (optimal solutions found from all the algorithms, all the runs)
combined_pf = get_combined_PF(dfs, drop_cols=relevant_cols);

In [ ]:
# Run this cell only once!!!
pfs = [unscale(pf, min_opening, 0.1, 0.05) for pf in pfs];

In [ ]:
# Sanity check!!!
first(pfs[1], 6)

In [ ]:
# Since sDA is actually a maximization, let us use the symmetric operation 
pfs = [get_symmetric(pf, sDA) for pf in pfs]
combined_pf = get_symmetric(combined_pf, sDA);

In [ ]:
S# Sort sDA values in ascending order

# Run this cell only once!!!

## This fixes the error we were getting in the create_pfs function,
## which would return a different point than the one we selected

pfs = [sort(pf, [sDA], rev=false) for pf in pfs];

In [ ]:
# Sanity check!!!
first(pfs[1], 6)

##  LEED V4.1 Daylight Credit

 To score in the LEED V4 Daylight Credit a solution must simultaneously meet the following requirements:
 
$$ ASE < 20\% \\ sDA \geq\ 40\% $$

### Pareto Front Plot

Choose the backend to visualize the Pareto front solutions:

In [ ]:
backend(autocad)

Create interactive Pareto front plot:

In [ ]:
create_pfs(pfs, x=sDA, y=ASE, tpf=nothing, names=all_algorithms, 
           colorscale=nothing, colors=["#43a0b5", "#B5557A", "#eb911c"],
           draw_dominated=true, layout=layout_tiling)

| Resulting Pareto front from NSGA-II optimization process.                      |
|------------------------------------------------------|
| <img src="./figures/pareto_front.png" width="1000"> | 

### Parallel Coordinates

To understand the impact the variables have in the overall performance, we also created a parallel coordinated plot and highlighted the solutions that score a LEED V4.1 Daylight Credit.

__Color scales:__ https://plotly.com/javascript/colorscales/

In [ ]:
# Code to manually define the color scale:
# colorscale=[(0,"red"), (0.5,"green"),(1,"blue")]),

In [ ]:
# Data frame used for the Parallel Coordinates plot
df = pfs[1];

Uncostrained:

In [ ]:
unconstrained_trace = parcoords(;line = attr(color=df.sDA, colorscale="YlGnBu"),
                      dimensions = [
                          attr(range = [0.1,0.85],
                               label = "min_opening [m]", values = df.min_opening),
                          attr(range = [10,50],
                               label = "n", values = df.n),
                          attr(range = [0,40],
                               label = "ASE [%]", values = df.ASE),
                          attr(range = [0,50],
                               label = "sDA [%]", values = df.sDA)
                      ]);

unconstrained_plot = plot(unconstrained_trace)

| Resulting Parallel Coordinates Plot from NSGA-II optimization process.|
|------------------------------------------------------|
| <img src="./figures/parallel_coord.png" width="900"> | 

Constrained:

In [ ]:
constrained_trace = parcoords(;line = attr(color=df.sDA, colorscale="YlGnBu"),
                    dimensions = [
                        attr(range = [0.1,0.85],
                               label = "min_opening [m]", values = df.min_opening),
                        attr(range = [10,50],
                               label = "n", values = df.n),
                        attr(range = [0,40], constraintrange = [0,20],
                               label = "ASE [%]", values = df.ASE),
                        attr(range = [0,50], constraintrange = [40,100],
                               label = "sDA [%]", values = df.sDA)
                    ]);

constrained_plot = plot(constrained_trace)

| Resulting Parallel Coordinates Plot from NSGA-II optimization process with restricted ASE and sDA values.|
|------------------------------------------------------|
| <img src="./figures/parallel_coord_restrict.png" width="900"> | 

###  Chosen Solution

__Non Dominated solutions resume:__

In [ ]:
# Filtered solutions: show non dominated
pfs_nd = filter(row -> row[:isDominated] == 0, pfs[1])[:, 1:4]

Expected result
<img src="./figures/non_dominated_chart.png" width="200">

Extract the parameters from the non-dominated solutions:

In [ ]:
sols_matrix=convert(Matrix, pfs_nd[:, 1:2])
non_dominated_sols=getindex.([sols_matrix], 1:size(sols_matrix, 1),:);

In [ ]:
# Background for Unity

function background_wall() 
    background_fam = wall_family_element(default_wall_family())
    set_backend_family(background_fam, unity, unity_material_family("Default/Materials/WhiteUnlit"))    
    wall([xy(-50, 0), xy(-50, 300)], bottom_level=level(-50), top_level=level(100), family = background_fam)
end   

The `show_solutions` function runs through the entire lists of provided parameter sets, generating the corresponding façade solutions in the backend for comparison. Each solution stays on screen for 4 seconds while the parameter values are printed.

In [ ]:
show_solutions(lst)=
 for vals in lst
    set_view(xyz(33.55,117.28,19.60), xyz(32.60,117.58,19.58), 30)
    delete_all_shapes()
    #     building(building_path, vals[1], n1=vals[end]) # view entire building structure with façade
    facade_tiles_vis(min1=vals[1], n1=vals[end]) # view façade only
    print(vals)
    sleep(4)
 end

The `save_solutions` function runs through the entire lists of provided parameter sets, generating the corresponding façade solutions in the backend. For each façade it saves a render image whose name corresponds to the parameter values.

In [ ]:
save_solutions(lst)=
 for vals in lst
    set_view(xyz(33.55,117.28,19.60), xyz(32.60,117.58,19.58), 30)
    delete_all_shapes()
    background_wall()
    facade_tiles_vis(min1=vals[1], n1=vals[end]) # view façade only
    render_view("$(vals[1])-$(vals[end])")
 end

In [ ]:
@test begin
    backend(unity)
    show_solutions(non_dominated_sols)
#     save_solutions(non_dominated_sols)
end

Expected result:

<img src="./figures/show_sols.png" width="1000">

After analyzing the non-dominated solutions, the following parameter sets were chosen:

In [ ]:
chosen_sols_params=[0.55, 34];

In [ ]:
@test begin
    backend(meshcat)
    render_size(800, 400)
    new_backend()
    end;

In [ ]:
@test begin
    building(building_path, chosen_sols_params[1], n1=chosen_sols_params[end])
#     facade_tiles_vis(min1=chosen_sols_params[1], n1=chosen_sols_params[end])
    end;

Expected result:

<img src="./figures/final_sol_meshcat.png" width="800">

###  Overview

Due to the conflicted nature of the performance goals, it was not possible to reconcile them, even when resorting to optimization strategies. The solutions found by the algorithm that yield an acceptable sDA value, also have a high ASE, meaning the wide availability of daylight will cause moments of visual discomfort. Nevertheless, the information provided by these visualizations is useful to guide future developments of the project, showing which variables the most impact in the performance, and how their ranges should be adjusted to achieve them.

In this project iteration, we opted for one of the non-dominated solutions in the Pareto Front, with one of the higher values for the number of tiles on the façade. However, better design solutions for the light problem will have to be sought. The sensitivity analysis results suggest we should also vary the size of the tiles acoording to the building's height: larger opening on the bottom to let the light in, and smaller ones on top to shade the interior. We plan on exploring this design option in the future. As the entire design-to-production process is automated in this Sketchbook, we have but to modify the initial design settings to almost instantly obtain the final models and plans for laser cutting.

#  Presentation

In [ ]:
@test begin
    backend(rhino)
#     backend(autocad)
#     backend(revit)
#     backend(unity)
end

In [ ]:
@test begin
    backend(povray)
    using Dates
    realistic_sky(turbidity=4,date=DateTime(2020, 9, 21, 8, 0, 0))
    render_size(1920, 1080)
end

In [ ]:
@test begin
    start_film("FaçadeOnly")  
    delete_all_shapes()
    facade_tiles_vis(building_path; min1=0.55, n1=34)
#     ground(z(0), rgb(0.1,0.1,0.1))
#     background_wall() 
    set_view(xyz(33.55,117.28,19.60), xyz(32.60,117.58,19.58), 30) # sideways façade view
#     save_film_frame()
end;

| Final Façade solution in AutoCAD  | Final Façade solution in Rhino | Final Façade solution in POV-Ray  |
|-----------------------------------|--------------------------------|-----------------------------------|
| <img src="./figures/presentation_autocad.png" width="300"> | <img src="./figures/presentation_rhino.png" width="320"> | <img src="./figures/presentation_POVRay.png" width="370"> | 


| Final Façade solution in Revit | Final Façade solution in Unity |
|--------------------------------|--------------------------------|
| <img src="./figures/presentation_revit.png" width="375"> | <img src="./figures/presentation_unity0.png" width="500"> |

Testing POV-Ray light and material setting options:

In [ ]:
@test begin
    set_backend_family(tile_fam, povray, povray_material_family(povray_material("Tile", gray=0.8)))
    set_backend_family(int_wall_fam, povray, povray_wall_family(povray_material("IntWall", gray=0.6)))
    set_backend_family(slab_fam, povray, povray_slab_family(povray_material("Slab", gray=0.6)))
    set_backend_family(roof_fam, povray, povray_roof_family(povray_material("Roof", gray=0.6)))
    realistic_sky(turbidity=4,date=DateTime(2020, 9, 21, 10, 0, 0))
    end;

In [ ]:
@test begin
    set_backend_family(tile_fam, povray, povray_material_family(povray_material("Tile", gray=0.8)))
    set_backend_family(int_wall_fam, povray, povray_wall_family(povray_material("IntWall", gray=0.8)))
    set_backend_family(slab_fam, povray, povray_slab_family(povray_material("Slab", gray=0.8)))
    set_backend_family(roof_fam, povray, povray_roof_family(povray_material("Roof", gray=0.9)))
    realistic_sky(turbidity=2,date=DateTime(2020, 9, 21, 8, 0, 0))
    end;

In [ ]:
@test begin
    delete_all_shapes()
    surroundings()
    building(building_path, 0.55, n1=34)
    start_film("FacadeTiling")
    
    set_view(xyz(-27.18,135.08,27.76), xyz(-26.29,135.53,27.68), 24) # indoors
    save_film_frame()
    
    set_view(xyz(-26.57,135.19,34.42), xyz(-25.67,135.62,34.43), 24)  # indoors last floor
    save_film_frame()
    
    set_view(xyz(-8.50321138,129.5714957,19.80033), xyz(-9.267270,130.214392,19.74650), 18) # façade left perspective
    save_film_frame()
    
    set_view(xyz(-13.8431126,144.6069496,0), xyz(-14.32287,144.519572,1), 18) # from the street up
    save_film_frame()
    
    set_view(xyz(21.4603,163.0398900,19.37334), xyz(20.527,162.679,19.356), 18) #other side of the street
    save_film_frame()
end;

Expected result:

|                                                            | POV-Ray Renders of Final Façade solution with surroundings.  |
|------------------------------------------------------------|--------------------------------------------------------------|
| <img src="./figures/FacadeTiling-frame-a.png" width="500"> | <img src="./figures/FacadeTiling-frame-002.png" width="500"> | 

|                                                            |                                                              |
|------------------------------------------------------------|--------------------------------------------------------------|
| <img src="./figures/FacadeTiling-frame-003.png" width="500"> | <img src="./figures/FacadeTiling-frame-004.png" width="500"> |

|                                                            |
|------------------------------------------------------------|
| <img src="./figures/FacadeTiling-frame-b.png" width="1000"> |

# Fabrication

The façade teling design is planned to be laser cut and assembled on site with metal hinges holding the plates in place. The algorithm bellow automatically generates the produce the different cutting plans in AutoCAD, suiting the specifications of the laser cutting machine. The file should indicate the cutting outlines of the tiles, along with their assembly order to facilitate their placement on the site.

|                                                            | Laser cutting panel subdivision scheme.  |
|------------------------------------------------------------|--------------------------------------------------------------|
| <img src="./figures/laser_1.png" width="450"> | <img src="./figures/laser_2.png" width="550"> | 

|                                                            |                                                              |
|------------------------------------------------------------|--------------------------------------------------------------|
| <img src="./figures/laser_4.png" width="450"> | <img src="./figures/laser_3.png" width="550"> |

New `facade_tiles` function creates separated tile panels maintaining the overall sinusoid effect:

In [ ]:
function facade_tiles(origin, direction, length, f_length, delta, height, min1=0.15, max1=0.9; n1=43, m=13)
    facade_layer = create_layer("Facade_Tiles")
    Khepri.with(current_layer, facade_layer) do
        Khepri.with(current_cs, cs_from_o_vx_vy(origin+vx(0.1), direction, vz(1))) do
            s1= itera_2triangs_mirrorXY(map_division(xy, 0, length, n1, 0, height, m))
            pattern.(polygonal_tile,
                     s1,
                     factor=pts->random_range(min1, max(min1, (abs ∘ sin)((pts[1][1].x+delta)/f_length*n_rooms*π)-(1-max1))))
        end
    end
end

New `building` function with façade tiling subdivided in a panel matrix for laser cutting:

In [ ]:
building_with_div_panels(slab_path=building_path, min1=0.55, max1=0.9, n1=34; horizontal_panels=4) =
    let pts = path_vertices(slab_path) 
        pt1, pt2, pt3, pte = pts[[1, 2, 3, end]]
        b1 = intermediate_loc(pt1, pte, 0.5)
        b2 = intermediate_loc(pt2, pt3, 0.5)  
        f_length = Khepri.distance(pts[1:2]...)
        
        tile_l = f_length / n1
        total_panel_h = default_level_to_level_height() * n_floors 
        f_height = total_panel_h + roof_fam.thickness      
        m = ceil(default_level_to_level_height() / tile_l)
    
        horiz_panel_length = f_length / horizontal_panels
        n_per_panel = ceil(horiz_panel_length / tile_l)
    
        normalize(x) = x/norm(x) 
        facade_vector = pt2-pt1
        panel_vector= normalize(facade_vector)*horiz_panel_length
    
        default_level(0)
        for i in 1:n_floors
        
            slab(slab_path, family=slab_fam) # slabs

            with_wall(slab_path) do w
              add_window(w, xy(0.1, 0.1), window_fam(f_length)) # contour walls + windows
            end

            wall([b1, b2], family=int_wall_fam) # interior back wall
        
            for ϵ in division(0, 1, n_rooms)[2:end-1] # interior office walls
                wall([intermediate_loc(b1, b2, ϵ), intermediate_loc(pt1, pt2, ϵ)], family=int_wall_fam)
            end
            
            for j in 0:horizontal_panels-1 # façade tile panels
                facade_tiles(pt1+vz(default_level().height)+panel_vector*j, 
                    panel_vector, horiz_panel_length, f_length, horiz_panel_length*j,
                    default_level_to_level_height(), 
                    min1, max1, n1=n_per_panel, m=m)
            end
        
            default_level(upper_level())
        end
        roof(slab_path, family=roof_fam) # roof slab
#         facade_tiles(pt1, pt2-pt1, f_length, f_height, min1, max1, n1=n1, m=m) # original façade tiles
    end

In [ ]:
@test begin
#     backend(rhino)
    backend(autocad)
end

In [ ]:
@test begin
    delete_all_shapes()
    building_with_div_panels(horizontal_panels=10)
#     set_view(xyz(33.55,117.28,19.60), xyz(32.60,117.58,19.58), 30) # sideways façade view
end;

`panels_4_laser` prints 2D planification of tile panels laser cutting:

In [ ]:
panels_4_laser(slab_path=building_path, min1=0.55, max1=0.9, n1=34; horizontal_panels=4) =
    let pts = path_vertices(slab_path) 
        pt1, pt2, pt3, pte = pts[[1, 2, 3, end]]
        f_length = Khepri.distance(pts[1:2]...)
        
        tile_l = f_length / n1
        total_panel_h = default_level_to_level_height() * n_floors 
        f_height = total_panel_h + roof_fam.thickness      
        m = ceil(default_level_to_level_height() / tile_l)
    
        horiz_panel_length = f_length / horizontal_panels
        n_per_panel = ceil(horiz_panel_length / tile_l)

        panel_vector= vx(horiz_panel_length)
        offset = tile_l*(1-max1)/2
    
        # draw 2D cutting lines only
        function facade_tiles(origin, direction, length, f_length, delta, height, min1=0.15, max1=0.9; n1=43, m=13)
    
            polygonal_tile(pts) =
                (; factor=0.5) ->
                let p = polygon_center(pts)
                    qts = [intermediate_loc(p,q,factor) for q in pts]
                    polygon(Khepri.distance(qts[1], p) < 0.05 ? [] : qts)
                end

            Khepri.with(current_cs, cs_from_o_vx_vy(origin, vx(), vy())) do
                s1= itera_2triangs_mirrorXY(map_division(xy, 0, length, n1, 0, height, m))
                pattern.(polygonal_tile,
                         s1,
                         factor=pts->random_range(min1, max(min1, (abs ∘ sin)((pts[1][1].x+delta)/f_length*n_rooms*π)-(1-max1))))
            end
        end
    
        default_level(0)
        for i in 1:n_floors           
            for j in 0:horizontal_panels-1 # façade tile panels
                let origin = pt1+vy(default_level().height)+panel_vector*j
                    facade_tiles(origin, 
                        panel_vector, horiz_panel_length, f_length, horiz_panel_length*j,
                        default_level_to_level_height(), 
                        min1, max1, n1=n_per_panel, m=m)
                    rectangle(origin, horiz_panel_length, default_level_to_level_height())
                    for p in [origin, origin+vx(horiz_panel_length-3offset)] 
                        text("$(i)/$(j+1)", p+vxy(offset/2, offset/2), offset/2)
                    end
                end
            end       
            default_level(upper_level())
        end
    end

In [ ]:
@test begin
    delete_all_shapes()
    panels_4_laser(horizontal_panels=4)
end;

| Laser cutting planification in AutoCAD  |
|------------------------------------------|
| <img src="./figures/autocad_laser_plan.png" width="500"> | 

| Panel number identification close-up |
|------------------------------------------|
|<img src="./figures/autocad_laser_id.png" width="900"> |

|    Panel number identification close-up |
|------------------------------------------|
|<img src="./figures/autocad_laser_id2.png" width="900"> |

_The end_